In [1]:
# import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark import SparkContext 

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.11:2.4.3 --conf spark.cassandra.connection.host=93.175.29.107,93.175.29.109,93.175.29.110,93.175.29.111,93.175.29.113,93.175.29.114,93.175.29.115,93.175.29.116 pyspark-shell'

In [3]:
SparkContext.getOrCreate().stop()

In [4]:
spark = SparkSession.builder \
    .appName("CassandraIntegration") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.11:2.4.3") \
    .config("spark.cassandra.connection.host", "93.175.29.107,93.175.29.109,93.175.29.110,93.175.29.111,93.175.29.113,93.175.29.114,93.175.29.115,93.175.29") \
    .config("spark.cassandra.connection.port", "9042") \
    .getOrCreate()

In [5]:
# List keyspaces to validate connection
def list_keyspaces():
    df = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(table="keyspaces", keyspace="system_schema") \
        .load()
    return df


In [6]:
print(list_keyspaces())

DataFrame[keyspace_name: string, durable_writes: boolean, replication: map<string,string>]


In [7]:
def load_data_from_cassandra(keyspace, table):
    df = spark.read \
        .format('org.apache.spark.sql.cassandra') \
        .options(table=table, keyspace=keyspace) \
        .load()
    return df

In [8]:
movies = load_data_from_cassandra('fedotov19', 'videos')

In [9]:
print(movies)

DataFrame[video_id: string, added_date: timestamp, description: string, title: string, user_id: string]


In [10]:
df_keyspaces = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="keyspaces", keyspace="system_schema") \
    .load()

df_keyspaces.show()

+--------------------+--------------+--------------------+
|       keyspace_name|durable_writes|         replication|
+--------------------+--------------+--------------------+
|hob2020034_hob202...|          true|[class -> org.apa...|
|          hob2020045|          true|[class -> org.apa...|
|    shkunkov_hw_pubg|          true|[class -> org.apa...|
|   babinolegkeyspace|          true|[class -> org.apa...|
|   msb202017keyspace|          true|[class -> org.apa...|
|             test123|          true|[class -> org.apa...|
|     chebotariv_test|          true|[class -> org.apa...|
|             dungeon|          true|[class -> org.apa...|
|            ttansh28|          true|[class -> org.apa...|
|          astanovaks|          true|[class -> org.apa...|
|           ache_test|          true|[class -> org.apa...|
|  akhmatdianov_aleks|          true|[class -> org.apa...|
|           hob201907|          true|[class -> org.apa...|
|          andresokol|          true|[class -> org.apa..